In [1]:
import cython
%load_ext cython

In [57]:
%%cython -f -a
# distutils: language = c++
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False

from cython.parallel cimport prange
from libc.math cimport cbrt
from libcpp.vector cimport vector
from libcpp cimport bool as cpp_bool

import numpy as np
cimport numpy as cnp
cnp.import_array()

from TidalPy.rheology.base cimport RheologyModelBase

def build_planet_constant_layers(
        double planet_radius,
        double forcing_frequency,
        
        tuple density_tuple,
        tuple static_bulk_modulus_tuple,
        tuple static_shear_modulus_tuple,
        tuple bulk_viscosity_tuple,
        tuple shear_viscosity_tuple,
        tuple layer_type_tuple,
        tuple layer_is_static_tuple,
        tuple layer_is_incompressible_tuple,
        tuple shear_rheology_model_tuple,
        tuple bulk_rheology_model_tuple,
        tuple radius_fraction_tuple = None,
        tuple volume_fraction_tuple = None,
        tuple slices_tuple = None,
        size_t slice_per_layer = 10,
        cpp_bool perform_checks = True):
    
    # Optimizations
    cdef double R3 = planet_radius**3
    
    # Convert to radius fractions
    cdef size_t layer_i
    cdef size_t num_layers = len(density_tuple)
    if perform_checks:
        assert len(static_bulk_modulus_tuple)   == num_layers
        assert len(static_shear_modulus_tuple)  == num_layers
        assert len(bulk_viscosity_tuple)        == num_layers
        assert len(shear_viscosity_tuple)       == num_layers
        assert len(layer_type_tuple)            == num_layers
        assert len(layer_is_static_tuple)       == num_layers
        assert len(layer_is_incompressible_tuple) == num_layers
        assert len(shear_rheology_model_tuple)  == num_layers
        assert len(bulk_rheology_model_tuple)   == num_layers

        if slices_tuple:
            assert len(slices_tuple) == num_layers

    cdef double rad_frac, last_layer_frac, vol_frac
    cdef list radius_fraction_list
    if radius_fraction_tuple is None:
        if volume_fraction_tuple is None:
            raise AttributeError("Must provide either `radius_fraction_tuple` or `volume_fraction_tuple`.")
        assert len(volume_fraction_tuple) == num_layers
        radius_fraction_list = list()
        last_layer_frac = 0.0
        for layer_i in range(num_layers):
            vol_frac = volume_fraction_tuple[layer_i]
            rad_frac = cbrt(vol_frac + last_layer_frac**3)
            radius_fraction_list.append(rad_frac)
            last_layer_frac = rad_frac
        radius_fraction_tuple = tuple(radius_fraction_list)
    elif perform_checks:
        assert len(radius_fraction_tuple) == num_layers
    
    cdef cnp.ndarray upper_radius_array = np.nan * np.ones(num_layers, dtype=np.float64, order='C')
    cdef double[::1] upper_radius_view  = upper_radius_array
    
    cdef vector[size_t] layer_top_index_vec = vector[size_t]()
    layer_top_index_vec.resize(num_layers)
    cdef double layer_thickness, layer_radius
    cdef double total_rad_frac = 0.0
    cdef size_t layer_slices = 0
    cdef size_t total_slices = 0
    for layer_i in range(num_layers):
        rad_frac = radius_fraction_tuple[layer_i]
        layer_thickness = rad_frac * planet_radius
        if rad_frac <= 0.0:
            raise AttributeError("Negative or zero radius fraction encountered.")
        total_rad_frac += rad_frac
        
        if slices_tuple:
            layer_slices = slices_tuple[layer_i]
        else:
            layer_slices = slice_per_layer
        if layer_i == 0:
            layer_top_index_vec[layer_i] = 0
            upper_radius_view[layer_i]   = layer_thickness
        else:
            layer_top_index_vec[layer_i] = layer_top_index_vec[layer_i - 1] + layer_slices
            upper_radius_view[layer_i]   = upper_radius_view[layer_i - 1] + layer_thickness
            
        if layer_slices < 5:
            raise AttributeError(f"Layer {layer_i} has {layer_slices} slices when at least 5 are required.")
        total_slices += layer_slices

        # Check if rheology classes are the correct instances
        if perform_checks:
            if not isinstance(shear_rheology_model_tuple[layer_i], RheologyModelBase):
                raise AttributeError(f"Layer {layer_i} shear rheology class is not an instance of `RheologyModelBase`.")
            if not isinstance(bulk_rheology_model_tuple[layer_i], RheologyModelBase):
                raise AttributeError(f"Layer {layer_i} shear rheology class is not an instance of `RheologyModelBase`.")
    
    if total_rad_frac != 1.0:
        raise ValueError(f"Unexpected value found for total radius fraction, {total_rad_frac} (expected 1.0).")
    
    # Build required arrays
    cdef cnp.ndarray radius_array          = np.nan * np.ones(total_slices, dtype=np.float64, order='C')
    cdef cnp.ndarray density_array         = np.nan * np.ones(total_slices, dtype=np.float64, order='C')
    cdef cnp.ndarray shear_viscosity_array = np.nan * np.ones(total_slices, dtype=np.float64, order='C')
    cdef cnp.ndarray bulk_viscosity_array  = np.nan * np.ones(total_slices, dtype=np.float64, order='C')
    cdef cnp.ndarray shear_array           = np.nan * np.ones(total_slices, dtype=np.float64, order='C')
    cdef cnp.ndarray bulk_array            = np.nan * np.ones(total_slices, dtype=np.float64, order='C')
    cdef cnp.ndarray complex_shear_array   = np.nan * np.ones(total_slices, dtype=np.complex128, order='C')
    cdef cnp.ndarray complex_bulk_array    = np.nan * np.ones(total_slices, dtype=np.complex128, order='C')

    cdef double* modulus_ptr                 = NULL
    cdef double* viscosity_ptr               = NULL
    cdef double complex* complex_modulus_ptr = NULL

    cdef double[::1] radius_view          = radius_array
    cdef double[::1] density_view         = density_array
    cdef double[::1] shear_viscosity_view = shear_viscosity_array
    cdef double[::1] bulk_viscosity_view  = bulk_viscosity_array
    cdef double[::1] shear_view           = shear_array
    cdef double[::1] bulk_view            = bulk_array
    cdef double complex[::1] complex_shear_view = complex_shear_array
    cdef double complex[::1] complex_bulk_view  = complex_bulk_array

    cdef RheologyModelBase rheo_instance

    cdef double planet_bulk_density = 0.0
    cdef double dr
    cdef size_t slice_i, full_index
    cdef double layer_density, layer_static_bulk, layer_static_shear, layer_bulk_visc, layer_shear_visc

    cdef size_t last_layer_top_slice = 0
    cdef double last_layer_radius = 0.0

    cdef Py_ssize_t sg_layer_i
    cdef Py_ssize_t sg_num_layers = <Py_ssize_t>num_layers
    
    for sg_layer_i in prange(sg_num_layers, nogil=True):

        if sg_layer_i == 0:
            last_layer_top_slice = 0
            last_layer_radius = 0.0
        else:
            last_layer_top_slice = layer_top_index_vec[sg_layer_i - 1]
            last_layer_radius = upper_radius_view[sg_layer_i - 1]
            
        if slices_tuple:
            layer_slices = slices_tuple[sg_layer_i]
        else:
            layer_slices = slice_per_layer
        rad_frac = radius_fraction_tuple[sg_layer_i]

        # Pull out other data
        layer_density      = density_tuple[sg_layer_i]
        layer_static_bulk  = static_bulk_modulus_tuple[sg_layer_i]
        layer_static_shear = static_shear_modulus_tuple[sg_layer_i]
        layer_bulk_visc    = bulk_viscosity_tuple[sg_layer_i]
        layer_shear_visc   = shear_viscosity_tuple[sg_layer_i]

        # Build linspace radius array from bottom of layer to top, inclusive of both r-bot and r-top
        layer_thickness = rad_frac * planet_radius
        layer_radius    = last_layer_radius + layer_thickness
        # The "- 1" in dr ensures that we build up to be inclusive of the top of the layer.
        # Earlier we ensured that layer_slices >= 5 so no worries about a divide by zero
        dr = layer_thickness / <double>(layer_slices - 1)

        # Other calculations
        upper_radius_view[layer_i] = layer_radius
        planet_bulk_density += layer_density * (layer_radius**3 - last_layer_radius**3)

        # Populate arrays
        for slice_i in range(layer_slices):
            full_index = last_layer_top_slice + slice_i
            # Fill in radius
            radius_view[full_index] = last_layer_radius + slice_i * dr
            # Fill in other arrays (This function assumes these properties are constant throughout the layer)
            density_view[full_index]         = layer_density
            shear_viscosity_view[full_index] = layer_shear_visc
            bulk_viscosity_view[full_index]  = layer_bulk_visc
            shear_view[full_index]           = layer_static_shear
            bulk_view[full_index]            = layer_static_bulk

        # Call on rheology class instance to find complex moduli.
        # Setup pointers used by rheology class
        modulus_ptr         = &shear_view[last_layer_top_slice]
        viscosity_ptr       = &shear_viscosity_view[last_layer_top_slice]
        complex_modulus_ptr = &complex_shear_view[last_layer_top_slice]
        rheo_instance = shear_rheology_model_tuple[sg_layer_i]
        rheo_instance._vectorize_modulus_viscosity(
            forcing_frequency,
            modulus_ptr,
            viscosity_ptr,
            complex_modulus_ptr,  # Modified variable
            <Py_ssize_t>layer_slices)
        
        # Repeat for bulk
        modulus_ptr         = &bulk_view[last_layer_top_slice]
        viscosity_ptr       = &bulk_viscosity_view[last_layer_top_slice]
        complex_modulus_ptr = &complex_bulk_view[last_layer_top_slice]
        rheo_instance = bulk_rheology_model_tuple[sg_layer_i]
        rheo_instance._vectorize_modulus_viscosity(
            forcing_frequency,
            modulus_ptr,
            viscosity_ptr,
            complex_modulus_ptr,  # Modified variable
            <Py_ssize_t>layer_slices)
        
    # Finalize other parameters
    planet_bulk_density /= R3 
    
    # Build outputs
    cdef tuple rs_outputs = (
        radius_array,
        density_array,
        complex_bulk_array,
        complex_shear_array,
        forcing_frequency,
        planet_bulk_density,
        layer_type_tuple,
        layer_is_static_tuple,
        layer_is_incompressible_tuple,
        upper_radius_array
    )

    return rs_outputs



Error compiling Cython file:
------------------------------------------------------------
...
        # Call on rheology class instance to find complex moduli.
        # Setup pointers used by rheology class
        modulus_ptr         = &shear_view[last_layer_top_slice]
        viscosity_ptr       = &shear_viscosity_view[last_layer_top_slice]
        complex_modulus_ptr = &complex_shear_view[last_layer_top_slice]
        rheo_instance = shear_rheology_model_tuple[sg_layer_i]
        ^
------------------------------------------------------------

C:\Users\joepr\.ipython\cython\_cython_magic_e4aa2adf5fcd57dafe3e37d59e28e3d214c13c26.pyx:199:8: Assignment of Python object not allowed without gil

Error compiling Cython file:
------------------------------------------------------------
...
        
        # Repeat for bulk
        modulus_ptr         = &bulk_view[last_layer_top_slice]
        viscosity_ptr       = &bulk_viscosity_view[last_layer_top_slice]
        complex_modulus_ptr = &

In [53]:
from TidalPy.rheology import Maxwell, Elastic, Newton

planet_radius=1.0e6
forcing_frequency=0.00001
density_list=tuple([11000., 9000., 4500.])
static_bulk_modulus_list=tuple([150.0e9, 100.0e9, 50.0e9])
static_shear_modulus_list=tuple([100.0e9, 0.0, 25.0e9])
bulk_viscosity_list=tuple([1.0e30, 1.0e28, 1.0e20])
shear_viscosity_list=tuple([1.0e27, 1000., 1.0e16])
layer_type_list=tuple(['solid', 'liquid', 'solid'])
layer_is_static_list=tuple([False, True, False])
layer_is_incompressible_list=tuple([False, False, False])
shear_rheology_model_list=tuple([Maxwell(), Newton(), Maxwell()])
bulk_rheology_model_list=tuple([Elastic(), Elastic(), Elastic()])
radius_fraction_list=tuple([0.25, 0.20, 1.0 - (0.25 + 0.2)])
volume_fraction_list=None
slices_list=None
slice_per_layer=10

out = build_planet_constant_layers(planet_radius,forcing_frequency,density_list,static_bulk_modulus_list,static_shear_modulus_list,bulk_viscosity_list,shear_viscosity_list,layer_type_list,layer_is_static_list,layer_is_incompressible_list,shear_rheology_model_list,bulk_rheology_model_list,radius_fraction_list,volume_fraction_list,slices_list,slice_per_layer)


In [54]:
out

(array([      0.        ,   27777.77777778,   55555.55555556,
          83333.33333333,  111111.11111111,  138888.88888889,
         166666.66666667,  194444.44444444,  222222.22222222,
         250000.        ,  250000.        ,  272222.22222222,
         294444.44444444,  316666.66666667,  338888.88888889,
         361111.11111111,  383333.33333333,  405555.55555556,
         427777.77777778,  450000.        ,  450000.        ,
         511111.11111111,  572222.22222222,  633333.33333333,
         694444.44444444,  755555.55555556,  816666.66666667,
         877777.77777778,  938888.88888889, 1000000.        ]),
 array([11000., 11000., 11000., 11000., 11000., 11000., 11000., 11000.,
        11000., 11000.,  9000.,  9000.,  9000.,  9000.,  9000.,  9000.,
         9000.,  9000.,  9000.,  9000.,  4500.,  4500.,  4500.,  4500.,
         4500.,  4500.,  4500.,  4500.,  4500.,  4500.]),
 array([1.5e+11+0.j, 1.5e+11+0.j, 1.5e+11+0.j, 1.5e+11+0.j, 1.5e+11+0.j,
        1.5e+11+0.j, 1.5e+11+0.

In [55]:
# 100us 96.4us
# 95us 94.4us
# 
%timeit build_planet_constant_layers(planet_radius,forcing_frequency,density_list,static_bulk_modulus_list,static_shear_modulus_list,bulk_viscosity_list,shear_viscosity_list,layer_type_list,layer_is_static_list,layer_is_incompressible_list,shear_rheology_model_list,bulk_rheology_model_list,radius_fraction_list,volume_fraction_list,slices_list,slice_per_layer, perform_checks=False)

110 μs ± 2.06 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
